<a href="https://colab.research.google.com/github/Santosh-Gupta/NaturalLanguageRecommendations/blob/srihari-dev/notebooks/tfrecords_debug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x

UsageError: Line magic function `%tensorflow_version` not found.


In [2]:
import tensorflow as tf
from tqdm import tqdm_notebook as tqdm
import os

print('TensorFlow:', tf.__version__)

TensorFlow: 2.0.0


In [3]:
batch_size  = 8
embedding_dim = 512
autotune = tf.data.experimental.AUTOTUNE

In [4]:
def get_random_title():
    return tf.random.uniform(shape=[512], maxval=200, dtype=tf.int32)

def get_random_citation():
    vector = tf.random.uniform(shape=[embedding_dim], minval=-1, maxval=1, dtype=tf.float32)
    normed_vector = tf.math.l2_normalize(vector)
    return normed_vector

def generate_sample():
    title = get_random_title()
    posCitations = get_random_citation()
    return title, posCitations

In [6]:
class TFrecordWriter:
    def __init__(self,
                 n_samples,
                 n_shards,
                 output_dir='',
                 prefix=''):
        self.n_samples = n_samples
        self.n_shards = n_shards
        self.step_size = self.n_samples//self.n_shards + 1
        self.prefix = prefix
        self.output_dir = output_dir
        self.buffer = []
        self.file_count = 1
        
    def make_example(self, title, vector):
        feature = {
            'title': tf.train.Feature(int64_list=tf.train.Int64List(value=title)),
            'citation': tf.train.Feature(float_list=tf.train.FloatList(value=vector))
        }
        return tf.train.Example(features=tf.train.Features(feature=feature))
        
    def write_tfrecord(self, tfrecord_path):
        print('writing {} samples in {}'.format(len(self.buffer), tfrecord_path))
        with tf.io.TFRecordWriter(tfrecord_path) as writer:
            for (title, vector) in tqdm(self.buffer):
                example = self.make_example(title, vector)
                writer.write(example.SerializeToString())
    
    def push(self, title, vector):
        self.buffer.append([title, vector])
        if len(self.buffer) == self.step_size:
            fname = self.prefix + '_000' + str(self.file_count) + '.tfrecord'
            tfrecord_path = os.path.join(self.output_dir, fname)
            self.write_tfrecord(tfrecord_path)
            self.clear_buffer()
            self.file_count += 1
            
    def flush_last(self):
        if len(self.buffer):
            fname = self.prefix + '_000' + str(self.file_count) + '.tfrecord'
            tfrecord_path = os.path.join(self.output_dir, fname)
            self.write_tfrecord(tfrecord_path)
            
    def clear_buffer(self):
        self.buffer = []

In [7]:
!mkdir 'tfrecords'
tfrecord_writer = TFrecordWriter(1000, 4, 'tfrecords', 'train')

In [8]:
for i in range(1000):
    title, vector = generate_sample()
    tfrecord_writer.push(title, vector)
tfrecord_writer.flush_last()

writing 251 samples in tfrecords/train_0001.tfrecord


/Users/srihari/tf2.0/lib/python3.6/site-packages/ipykernel_launcher.py:25: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



writing 251 samples in tfrecords/train_0002.tfrecord



writing 251 samples in tfrecords/train_0003.tfrecord



writing 247 samples in tfrecords/train_0004.tfrecord
